In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import warnings
import datetime
import argparse
import threading
import multiprocessing
import os

from tqdm import tqdm

from GBP_Simulations.GBP.data import DataGenerator
from GBP_Simulations.GBP.gbp import run_GaBP_SYNC_ACCELERATED, run_GaBP_HARDWARE_ACCELERATED, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_MAX, run_GaBP_HARDWARE_ACCELERATED_EXCLUSION, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_nP
from GBP_Simulations.GBP.utilities import HiddenPrints
from GBP_Simulations.GBP.visulisation import set_plot_options, get_plot_colors, NetworkxGraph, AnalyzeResult

# Option 1: Suppress all warnings
warnings.filterwarnings("ignore")

set_plot_options()
colors = get_plot_colors()

data_gen = DataGenerator()
result_analyzer = AnalyzeResult()

In [ ]:
# file_path = 'GBP_Simulations/GBP/Raw_Datasets/data/input_MITb_g2o.g2o'
# data_gen.generate_SLAM_dataset(file_path=file_path)

In [ ]:




dataset = 'input_MITb_g2o'
filepath_n = 'GBP_Simulations/GBP/Raw_Datasets/gbp_data'
factor_data = os.path.join(filepath_n, f'{dataset}_factor_data.txt')
marginal_data = os.path.join(filepath_n, f'{dataset}_marginal_data.txt')



A, b = data_gen.fetch_SLAM_dataset(file_path_factor=factor_data, file_path_marginal=marginal_data)


filename = "slam_mitb"
with open(f"Hardware_Model/designs/{filename}.json") as f:
    data = json.load(f)

# Extract designs
graph = data.get('inp_graph_topology', [])
designs = data.get('all_designs', [])





In [ ]:
designs

In [ ]:
limit = 100
design_with_max_nodes_on_chip = None
for design in designs:
    resources_percent = design['resources']['resources_%']
    if resources_percent['lut'] < limit and resources_percent['ff'] < limit and resources_percent['bram'] < limit and resources_percent['dsp'] < limit:
        if design_with_max_nodes_on_chip is None:
            design_with_max_nodes_on_chip = design
        else:
            node_update_per_second_previous = design_with_max_nodes_on_chip['design']['nodes_updt_per_stream']/design_with_max_nodes_on_chip['latency']['latency_total'] 
            node_update_per_second_new = design['design']['nodes_updt_per_stream']/design['latency']['latency_total']
            if node_update_per_second_new > node_update_per_second_previous:
                design_with_max_nodes_on_chip = design
print(design_with_max_nodes_on_chip)
print(design_with_max_nodes_on_chip['design']['nodes_updt_per_stream']/design_with_max_nodes_on_chip['latency']['latency_total'] )

In [ ]:
limits = [30,40,50,60,70,80,90,100]
ALL_DESIGNS = {}

for limit in limits:
    design_with_max_nodes_on_chip = None
    for design in designs:
        resources_percent = design['resources']['resources_%']
        if resources_percent['lut'] < limit and resources_percent['ff'] < limit and resources_percent['bram'] < limit and resources_percent['dsp'] < limit:
            if design_with_max_nodes_on_chip is None:
                design_with_max_nodes_on_chip = design
            else:
                node_update_per_second_previous = design_with_max_nodes_on_chip['design']['nodes_updt_per_stream']/design_with_max_nodes_on_chip['latency']['latency_total'] 
                node_update_per_second_new = design['design']['nodes_updt_per_stream']/design['latency']['latency_total']
                if node_update_per_second_new > node_update_per_second_previous:
                    design_with_max_nodes_on_chip = design
    
    if design_with_max_nodes_on_chip is not None:
        n = int(design_with_max_nodes_on_chip['design']['nodes_updt_per_pe'])
        p = int(design_with_max_nodes_on_chip['design']['number_pes'])
        ALL_DESIGNS[limit] = {'n':n, 'p':p, 'latency_total': design_with_max_nodes_on_chip['latency']['latency_total'], 'Rcf': n*p/design_with_max_nodes_on_chip['latency']['latency_total']}

In [ ]:
ALL_DESIGNS

# What is the best configuration?

### Sync

In [ ]:
sync_convergence_threshold = 1*10**-8 # convergence threshold
convergence_type = 'all' #all
simulation_convergence_threshold = 1*10**-8

P_i, mu_i, N_i, P_ii, mu_ii, P_ij, mu_ij, iter_dist, stand_divs, means, iteration = run_GaBP_SYNC_ACCELERATED(A, 
                                                                                                      b, 
                                                                                                      max_iter=100_000, 
                                                                                                      mae=True if convergence_type == 'mae' else False, 
                                                                                                      convergence_threshold=simulation_convergence_threshold,
                                                                                                      show=True)
final_mean = list(mu_i)

### Async

In [ ]:
print(ALL_DESIGNS)

In [ ]:
async_convergence_threshold = 1*10**-5
ASYNC_ITER = 1

ALL_DESIGNS_LATENCY = ALL_DESIGNS

In [ ]:
total_iterations = len(['residual-TnP-PREV']) * len([True]) * ASYNC_ITER * len(ALL_DESIGNS)

In [ ]:
pbar = tqdm(total=total_iterations, desc="Running GBP Simulations", unit="designs")

for limit, design in ALL_DESIGNS.items():
    NODE_UPDT_PE = design['n']
    PEs = design['p']

    RESULTS = {}
    PADD_RESULTS = {}

    for policy in ['residual-TnP-PREV']:
        for cache in [True]:

            str = f'{policy}_{cache}'

            # variable
            sum_of_iterations = 0

            sum_of_padding_ave = 0
            
            print(f"policy: {policy}, limit = {limit}, design: (n={NODE_UPDT_PE},P={PEs})")

            # implement multiple iterations
            it = 0
            while (it < ASYNC_ITER):
                if policy == 'fixed':
                    num_nodes = A.shape[0]
                    node_update_schedule = np.arange(num_nodes, dtype=np.int64)
                    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(
                        A, 
                        b, 
                        caching=True, 
                        mode='fixed', 
                        node_update_schedule_enter=node_update_schedule, 
                        node_updates_per_pe=NODE_UPDT_PE, 
                        number_pes=PEs, 
                        TRUE_MEAN=final_mean, 
                        max_iter=1500, 
                        mae=True, 
                        convergence_threshold=async_convergence_threshold, 
                        show=False)

                    if iteration < 1500:
                        sum_of_iterations += iteration
                        it += 1
                    else:
                        iteration = float("inf")
                        print("=========== FIXED: NOT CONVERGING ===========")
                elif policy == 'random':
                    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(
                        A, 
                        b, 
                        node_updates_per_pe=NODE_UPDT_PE, 
                        number_pes=PEs, 
                        TRUE_MEAN=final_mean, 
                        max_iter=1500, 
                        mae=True, 
                        convergence_threshold=async_convergence_threshold,
                        show=False,
                        mode=policy,
                        caching=cache
                    )
                    if iteration < 1500:
                        sum_of_iterations += iteration
                        it += 1
                    else:
                        iteration = float("inf")
                        print("=========== RANDOM-EXCLUSION: NOT CONVERGING ===========")
                elif policy == 'random-exclusion':
                    P_i, mu_i, iteration, _, _ = run_GaBP_HARDWARE_ACCELERATED_EXCLUSION(
                        A, 
                        b, 
                        caching=cache,
                        node_updates_per_pe=NODE_UPDT_PE, 
                        number_pes=PEs, 
                        TRUE_MEAN=final_mean, 
                        max_iter=1500, 
                        mae=True, 
                        convergence_threshold=async_convergence_threshold, 
                        show=False
                    )
                    if iteration < 1500:
                        sum_of_iterations += iteration
                        it += 1
                    else:
                        iteration = float("inf")
                        print("=========== RANDOM: NOT CONVERGING ===========")
                elif policy == 'residual':
                    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL(
                        A, 
                        b, 
                        caching=cache,
                        node_updates_per_pe=NODE_UPDT_PE, 
                        number_pes=PEs, 
                        TRUE_MEAN=final_mean, 
                        max_iter=1500, 
                        mae=True, 
                        convergence_threshold=async_convergence_threshold, 
                        show=False
                    )
                    if iteration < 1500:
                        sum_of_iterations += iteration
                        it += 1
                    else:
                        iteration = float("inf")
                        print("=========== RESIDUAL: NOT CONVERGING ===========")
                
                
                
                
                
                elif policy == 'residual-TMAX-PREV-scaling':
                    P_i, mu_i, iteration, _, PADDING_ANALYTICS = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_MAX(
                        A, 
                        b, 
                        SCALING=True,
                        SHUFFLE_STREAM=False, 
                        CURR_T_MAX=False, 
                        caching=True, 
                        node_updates_per_pe=NODE_UPDT_PE, 
                        number_pes=PEs, 
                        TRUE_MEAN=final_mean, 
                        max_iter=1500, 
                        mae=True, 
                        convergence_threshold=async_convergence_threshold, 
                        show=False
                    )
                    if iteration < 1500:
                        sum_of_iterations += iteration
                        sum_of_padding_ave += sum(PADDING_ANALYTICS)/len(PADDING_ANALYTICS)
                        it += 1
                    else:
                        iteration = float("inf")
                        print("=========== RESIDUAL: NOT CONVERGING ===========")
                elif policy == 'residual-TMAX-CURR-scaling':
                    P_i, mu_i, iteration, _, PADDING_ANALYTICS = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_MAX(
                        A, 
                        b,
                        SCALING=True,
                        SHUFFLE_STREAM=False, 
                        CURR_T_MAX=True, 
                        caching=True, 
                        node_updates_per_pe=NODE_UPDT_PE, 
                        number_pes=PEs, 
                        TRUE_MEAN=final_mean, 
                        max_iter=1500, 
                        mae=True, 
                        convergence_threshold=async_convergence_threshold, 
                        show=False
                    )
                    if iteration < 1500:
                        sum_of_iterations += iteration
                        sum_of_padding_ave += sum(PADDING_ANALYTICS)/len(PADDING_ANALYTICS)
                        it += 1
                    else:
                        iteration = float("inf")
                        print("=========== RESIDUAL: NOT CONVERGING ===========")
                elif policy == 'residual-TnP-PREV-scaling':
                    P_i, mu_i, iteration, _, PADDING_ANALYTICS = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_nP(
                        A, 
                        b,
                        SCALING=True,
                        SHUFFLE_STREAM=False, 
                        CURR_T_MAX=False, 
                        caching=True, 
                        node_updates_per_pe=NODE_UPDT_PE, 
                        number_pes=PEs, 
                        TRUE_MEAN=final_mean, 
                        max_iter=1500, 
                        mae=True, 
                        convergence_threshold=async_convergence_threshold, 
                        show=False
                    )
                    if iteration < 1500:
                        sum_of_iterations += iteration
                        sum_of_padding_ave += sum(PADDING_ANALYTICS)/len(PADDING_ANALYTICS)
                        it += 1
                    else:
                        iteration = float("inf")
                        print("=========== RESIDUAL: NOT CONVERGING ===========")
                elif policy == 'residual-TnP-CURR-scaling':
                    P_i, mu_i, iteration, _, PADDING_ANALYTICS = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_nP(
                        A, 
                        b, 
                        SCALING=True,
                        SHUFFLE_STREAM=False, 
                        CURR_T_MAX=True, 
                        caching=True, 
                        node_updates_per_pe=NODE_UPDT_PE, 
                        number_pes=PEs, 
                        TRUE_MEAN=final_mean, 
                        max_iter=1500, 
                        mae=True, 
                        convergence_threshold=async_convergence_threshold, 
                        show=False
                    )
                    if iteration < 1500:
                        sum_of_iterations += iteration
                        sum_of_padding_ave += sum(PADDING_ANALYTICS)/len(PADDING_ANALYTICS)
                        it += 1
                    else:
                        iteration = float("inf")
                        print("=========== RESIDUAL: NOT CONVERGING ===========")
                
                
                
                
                
                
                
                
                
                elif policy == 'residual-TMAX-PREV':
                    P_i, mu_i, iteration, _, PADDING_ANALYTICS = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_MAX(
                        A, 
                        b, 
                        SCALING=False,
                        SHUFFLE_STREAM=False, 
                        CURR_T_MAX=False, 
                        caching=True, 
                        node_updates_per_pe=NODE_UPDT_PE, 
                        number_pes=PEs, 
                        TRUE_MEAN=final_mean, 
                        max_iter=1500, 
                        mae=True, 
                        convergence_threshold=async_convergence_threshold, 
                        show=False
                    )
                    if iteration < 1500:
                        sum_of_iterations += iteration
                        sum_of_padding_ave += sum(PADDING_ANALYTICS)/len(PADDING_ANALYTICS)
                        it += 1
                    else:
                        iteration = float("inf")
                        print("=========== RESIDUAL: NOT CONVERGING ===========")
                elif policy == 'residual-TMAX-CURR':
                    P_i, mu_i, iteration, _, PADDING_ANALYTICS = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_MAX(
                        A, 
                        b,
                        SCALING=False,
                        SHUFFLE_STREAM=False, 
                        CURR_T_MAX=True, 
                        caching=True, 
                        node_updates_per_pe=NODE_UPDT_PE, 
                        number_pes=PEs, 
                        TRUE_MEAN=final_mean, 
                        max_iter=1500, 
                        mae=True, 
                        convergence_threshold=async_convergence_threshold, 
                        show=False
                    )
                    if iteration < 1500:
                        sum_of_iterations += iteration
                        sum_of_padding_ave += sum(PADDING_ANALYTICS)/len(PADDING_ANALYTICS)
                        it += 1
                    else:
                        iteration = float("inf")
                        print("=========== RESIDUAL: NOT CONVERGING ===========")
                elif policy == 'residual-TnP-PREV':
                    P_i, mu_i, iteration, _, PADDING_ANALYTICS = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_nP(
                        A, 
                        b,
                        SCALING=False,
                        SHUFFLE_STREAM=False, 
                        CURR_T_MAX=False, 
                        caching=True, 
                        node_updates_per_pe=NODE_UPDT_PE, 
                        number_pes=PEs, 
                        TRUE_MEAN=final_mean, 
                        max_iter=1500, 
                        mae=True, 
                        convergence_threshold=async_convergence_threshold, 
                        show=False
                    )
                    if iteration < 1500:
                        sum_of_iterations += iteration
                        sum_of_padding_ave += sum(PADDING_ANALYTICS)/len(PADDING_ANALYTICS)
                        it += 1
                    else:
                        iteration = float("inf")
                        print("=========== RESIDUAL: NOT CONVERGING ===========")
                elif policy == 'residual-TnP-CURR':
                    P_i, mu_i, iteration, _, PADDING_ANALYTICS = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_nP(
                        A, 
                        b,
                        SCALING=False,
                        SHUFFLE_STREAM=False, 
                        CURR_T_MAX=True, 
                        caching=True, 
                        node_updates_per_pe=NODE_UPDT_PE, 
                        number_pes=PEs, 
                        TRUE_MEAN=final_mean, 
                        max_iter=1500, 
                        mae=True, 
                        convergence_threshold=async_convergence_threshold, 
                        show=False
                    )
                    if iteration < 1500:
                        sum_of_iterations += iteration
                        sum_of_padding_ave += sum(PADDING_ANALYTICS)/len(PADDING_ANALYTICS)
                        it += 1
                    else:
                        iteration = float("inf")
                        print("=========== RESIDUAL: NOT CONVERGING ===========")        
                else:
                    # print(policy)
                    # print("----------------")
                    raise Exception("Error: No Matching Policy")              

                pbar.update(1)  

                # print(f"-------------- ITERATION = {it} => Streams = {iteration} --------------")
            
            # average
            ave = sum_of_iterations/ASYNC_ITER

            # padding average
            pad_ave = sum_of_padding_ave/ASYNC_ITER

            RESULTS[str] = ave

            PADD_RESULTS[str] = pad_ave

            ALL_DESIGNS_LATENCY[limit]['stream_results'] = RESULTS

            ALL_DESIGNS_LATENCY[limit]['padding_results'] = PADD_RESULTS

            # print("-----------")
pbar.close()

In [ ]:
ALL_DESIGNS

In [ ]:
# filename_to_save_design = f"Results/SLAM_MITb_residual_test_mae_extended.json"

# # Write the list of dictionaries to the file
# with open(filename_to_save_design, 'w') as file:
#     json.dump(ALL_DESIGNS, file, indent=4)